In [1]:
import pandas as pd
df = pd.read_csv("/kaggle/input/vr-baseline-mplug/evaluation_results.csv")
df.head()

,GroundTruth,Prediction,CleanedPrediction,CleanedGroundTruth,Correct
0,no,Yes,yes,no,False
1,no,No,no,no,True
2,yes,Yes,yes,yes,True
3,yes,Yes,yes,yes,True
4,yes,Yes,yes,yes,True


In [3]:
! pip install git+https://github.com/google-research/bleurt.git
!pip install scikit-learn evaluate bert-score rouge-score \
            sentence-transformers rapidfuzz

from bert_score import BERTScorer 
from rouge_score import rouge_scorer 
from rapidfuzz.distance import Levenshtein 
from sentence_transformers import SentenceTransformer, util 

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-te88r0u6
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-te88r0u6
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=cb17491f20394e1e70d2dd4e648061dedc56349fca25b3593206262b41e7fd67
  Stored in directory: /tmp/pip-ephem-wheel-cache-821rgzus/wheels/30/af/34/e148007788b060e4c76e7ecf68e70c692dff0f2632e62ac454
Successfully built BLEURT
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2025-05-18 07:16:52.035083: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747552612.308668      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747552612.391077      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from rouge_score import rouge_scorer
from bert_score import BERTScorer
# import Levenshtein
# from sentence_transformers import SentenceTransformer, util
import torch

# assume df is your DataFrame
preds_l = df["CleanedPrediction"].astype(str).str.lower().str.strip().tolist()
refs_l  = df["CleanedGroundTruth"].astype(str).str.lower().str.strip().tolist()

# 1) Exact-match
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(refs_l)
acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)
print(f"Exact-match  Acc: {acc:.3f}  Prec: {prec:.3f}  Rec: {rec:.3f}  F1: {f1:.3f}\n")

# 2) ROUGE
print("--- ROUGE Scores ---")
rouge = rouge_scorer.RougeScorer(["rouge1","rougeL"], use_stemmer=True)
r1, rL = [], []
for p, r in zip(preds_l, refs_l):
    if not p or not r:
        r1.append(0.0); rL.append(0.0)
    else:
        sc = rouge.score(r, p)
        r1.append(sc["rouge1"].fmeasure)
        rL.append(sc["rougeL"].fmeasure)
print(f"Avg ROUGE-1 F1: {np.mean(r1):.3f}")
print(f"Avg ROUGE-L F1: {np.mean(rL):.3f}\n")

# 3) BERTScore
print("--- BERTScore ---")
device = "cuda" if torch.cuda.is_available() else "cpu"
bs = BERTScorer(lang="en", rescale_with_baseline=True, device=device)
# filter out any empty pairs
valid = [(p,r) for p,r in zip(preds_l, refs_l) if p and r]
if valid:
    P, R, F = bs.score([p for p,_ in valid], [r for _,r in valid])
    print(f"Avg BERTScore P: {P.mean():.3f}  R: {R.mean():.3f}  F1: {F.mean():.3f}\n")
else:
    print("No valid pairs for BERTScore\n")

# 4) Levenshtein normalized similarity
print("--- Levenshtein Normalized Similarity ---")
lev = []
for p, r in zip(preds_l, refs_l):
    if not p and not r:
        lev.append(1.0)
    elif not p or not r:
        lev.append(0.0)
    else:
        lev.append(Levenshtein.normalized_similarity(p, r))
print(f"Avg Levenshtein sim: {np.mean(lev):.3f}\n")

# 5) Sentence-BERT cosine similarity
print("--- Sentence-BERT Cosine Similarity ---")
sbert = SentenceTransformer("all-MiniLM-L6-v2", device=device)
valid_idx = [i for i,(p,r) in enumerate(zip(preds_l, refs_l)) if p and r]
if valid_idx:
    sp = [preds_l[i] for i in valid_idx]
    sr = [refs_l[i]  for i in valid_idx]
    emb_p = sbert.encode(sp, convert_to_tensor=True)
    emb_r = sbert.encode(sr, convert_to_tensor=True)
    sims = util.cos_sim(emb_p, emb_r).diagonal().cpu().numpy()
    print(f"Avg SBERT cosine sim: {sims.mean():.3f}")
else:
    print("No valid pairs for SBERT similarity")


Exact-match  Acc: 0.610  Prec: 1.000  Rec: 0.610  F1: 0.758

--- ROUGE Scores ---
Avg ROUGE-1 F1: 0.627
Avg ROUGE-L F1: 0.627

--- BERTScore ---


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Batches:   0%|          | 0/313 [00:00<?, ?it/s]

Avg SBERT cosine sim: 0.817
